In [1]:
from collections import OrderedDict
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import sampler
import os
import torchvision.datasets as dset
import torchvision.transforms as T
import pandas as pd
# for plotting
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

from trainer_functions.cifartrainer import evaluate_cifar, train_cifar, build_cifar10
from models.ResNet import ResNetCifar as ResNet
from trainer_functions.cifarCloader import CIFARCDataset
from trainer_functions.imagenetc import ImageNetC
from trainer_functions.imagenetctrainer import build_resnet50, evaluate_imagenetc, train_imagenetc

In [2]:
# empty dataframe
df = pd.DataFrame(columns=['corruption', 'accuracy', "lr", "state", "n_subset"])

# corruption_list = ["brightness", "contrast", "defocus_blur", "elastic_transform", "fog", "frost", "gaussian_noise", "glass_blur",
# "impulse_noise", "jpeg_compression", "motion_blur", "pixelate", "shot_noise", "snow", "zoom_blur"]

corruption_list = ["frost", "gaussian_noise", "glass_blur", "impulse_noise", "snow"]

n_subset = 2000
lrs = [1e-3, 1e-4, 1e-5]

## writing fine tuning code here

In [3]:
# four layers to train

for corruption in corruption_list:
    for lr in lrs:
        dataset = ImageNetC(root='/Users/rada/Desktop/KAIST/Deep Learning/Datasets/ImageNetC',
                        corruption=corruption, severity=5,
                        transform=T.Compose([T.ToTensor()])
                        )

        indices = torch.randperm(len(dataset))

        ictrain = torch.utils.data.Subset(dataset, indices[:n_subset])
        icvalid = torch.utils.data.Subset(dataset, indices[n_subset:n_subset+2000])
        ictest = torch.utils.data.Subset(dataset, indices[n_subset+2000:n_subset+4000])

        ictrainload = DataLoader(ictrain, batch_size=64, shuffle=True)
        icvalidload = DataLoader(icvalid, batch_size=64, shuffle=True)
        ictestload = DataLoader(ictest, batch_size=64, shuffle=True)

        
        device = torch.device("mps")
        tune_net = build_resnet50(device)

        # tune_net.layer1 to tune the first layer, tune_net to tune all layers, tune_net.fc to tune the last year
        optimizer = optim.Adam(tune_net.layer1.parameters(), lr=lr, weight_decay=0.0001)

        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
        
        # checkpoint = torch.load('checkpoints/ckpt-2.pth')
        # tune_net.load_state_dict(checkpoint['net'])

        acc = train_imagenetc(tune_net, ictrainload, icvalidload, optimizer, scheduler, device=device, epochs=10)

        # adding results to dataframe
        dict = {}
        dict['corruption'] = corruption
        dict['accuracy'] = acc
        dict['lr'] = lr
        dict['state'] = "first layer" # change when run
        dict['n_subset'] = n_subset
        df_temp = pd.DataFrame(dict, index=[0])
        df = pd.concat([df, df_temp])

# change when run
df.to_csv("imageNetC_layer1.csv")

Building model...
Epoch 0, loss = 3.2286, lr 0.00100000
Got 919 / 2000 correct (45.95)

Epoch 1, loss = 2.0393, lr 0.00100000
Got 973 / 2000 correct (48.65)

Epoch 2, loss = 2.8932, lr 0.00100000
Got 1017 / 2000 correct (50.85)

Epoch 3, loss = 3.0702, lr 0.00100000
Got 1005 / 2000 correct (50.25)

Epoch 4, loss = 1.9675, lr 0.00100000
Got 1008 / 2000 correct (50.40)

Epoch 5, loss = 1.7241, lr 0.00100000
Got 994 / 2000 correct (49.70)

Epoch 6, loss = 2.6093, lr 0.00100000
Got 1027 / 2000 correct (51.35)

Epoch 7, loss = 2.2146, lr 0.00100000
Got 1011 / 2000 correct (50.55)

Epoch 8, loss = 2.1076, lr 0.00100000
Got 1000 / 2000 correct (50.00)

Epoch 9, loss = 1.3700, lr 0.00100000
Got 1006 / 2000 correct (50.30)

Building model...
Epoch 0, loss = 3.1209, lr 0.00010000
Got 911 / 2000 correct (45.55)

Epoch 1, loss = 3.2269, lr 0.00010000
Got 974 / 2000 correct (48.70)

Epoch 2, loss = 3.4049, lr 0.00010000
Got 978 / 2000 correct (48.90)

Epoch 3, loss = 3.6676, lr 0.00010000
Got 993 /

In [4]:
# four layers to train

for corruption in corruption_list:
    for lr in lrs:
        dataset = ImageNetC(root='/Users/rada/Desktop/KAIST/Deep Learning/Datasets/ImageNetC',
                        corruption=corruption, severity=5,
                        transform=T.Compose([T.ToTensor()])
                        )

        indices = torch.randperm(len(dataset))

        ictrain = torch.utils.data.Subset(dataset, indices[:n_subset])
        icvalid = torch.utils.data.Subset(dataset, indices[n_subset:n_subset+2000])
        ictest = torch.utils.data.Subset(dataset, indices[n_subset+2000:n_subset+4000])

        ictrainload = DataLoader(ictrain, batch_size=64, shuffle=True)
        icvalidload = DataLoader(icvalid, batch_size=64, shuffle=True)
        ictestload = DataLoader(ictest, batch_size=64, shuffle=True)

        
        device = torch.device("mps")
        tune_net = build_resnet50(device)

        # tune_net.layer1 to tune the first layer, tune_net to tune all layers, tune_net.fc to tune the last year
        optimizer = optim.Adam(tune_net.layer2.parameters(), lr=lr, weight_decay=0.0001)

        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
        
        # checkpoint = torch.load('checkpoints/ckpt-2.pth')
        # tune_net.load_state_dict(checkpoint['net'])

        acc = train_imagenetc(tune_net, ictrainload, icvalidload, optimizer, scheduler, device=device, epochs=10)

        # adding results to dataframe
        dict = {}
        dict['corruption'] = corruption
        dict['accuracy'] = acc
        dict['lr'] = lr
        dict['state'] = "second layer" # change when run
        dict['n_subset'] = n_subset
        df_temp = pd.DataFrame(dict, index=[0])
        df = pd.concat([df, df_temp])

# change when run
df.to_csv("imageNetC_layer2.csv")

Building model...
Epoch 0, loss = 3.0443, lr 0.00100000
Got 939 / 2000 correct (46.95)

Epoch 1, loss = 2.5874, lr 0.00100000
Got 1008 / 2000 correct (50.40)

Epoch 2, loss = 1.6024, lr 0.00100000
Got 1018 / 2000 correct (50.90)

Epoch 3, loss = 1.5784, lr 0.00100000
Got 1016 / 2000 correct (50.80)

Epoch 4, loss = 1.2513, lr 0.00100000
Got 1036 / 2000 correct (51.80)

Epoch 5, loss = 0.9280, lr 0.00100000
Got 997 / 2000 correct (49.85)

Epoch 6, loss = 1.5905, lr 0.00100000
Got 1000 / 2000 correct (50.00)

Epoch 7, loss = 1.2988, lr 0.00100000
Got 1030 / 2000 correct (51.50)

Epoch 8, loss = 0.4678, lr 0.00100000
Got 1012 / 2000 correct (50.60)

Epoch 9, loss = 0.6856, lr 0.00100000
Got 999 / 2000 correct (49.95)

Building model...
Epoch 0, loss = 2.8530, lr 0.00010000
Got 974 / 2000 correct (48.70)

Epoch 1, loss = 2.9863, lr 0.00010000
Got 1029 / 2000 correct (51.45)

Epoch 2, loss = 3.3689, lr 0.00010000
Got 1037 / 2000 correct (51.85)

Epoch 3, loss = 2.6663, lr 0.00010000
Got 104

In [5]:
# four layers to train

for corruption in corruption_list:
    for lr in lrs:
        dataset = ImageNetC(root='/Users/rada/Desktop/KAIST/Deep Learning/Datasets/ImageNetC',
                        corruption=corruption, severity=5,
                        transform=T.Compose([T.ToTensor()])
                        )

        indices = torch.randperm(len(dataset))

        ictrain = torch.utils.data.Subset(dataset, indices[:n_subset])
        icvalid = torch.utils.data.Subset(dataset, indices[n_subset:n_subset+2000])
        ictest = torch.utils.data.Subset(dataset, indices[n_subset+2000:n_subset+4000])

        ictrainload = DataLoader(ictrain, batch_size=64, shuffle=True)
        icvalidload = DataLoader(icvalid, batch_size=64, shuffle=True)
        ictestload = DataLoader(ictest, batch_size=64, shuffle=True)

        
        device = torch.device("mps")
        tune_net = build_resnet50(device)

        # tune_net.layer1 to tune the first layer, tune_net to tune all layers, tune_net.fc to tune the last year
        optimizer = optim.Adam(tune_net.layer3.parameters(), lr=lr, weight_decay=0.0001)

        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
        
        # checkpoint = torch.load('checkpoints/ckpt-2.pth')
        # tune_net.load_state_dict(checkpoint['net'])

        acc = train_imagenetc(tune_net, ictrainload, icvalidload, optimizer, scheduler, device=device, epochs=10)

        # adding results to dataframe
        dict = {}
        dict['corruption'] = corruption
        dict['accuracy'] = acc
        dict['lr'] = lr
        dict['state'] = "third layer" # change when run
        dict['n_subset'] = n_subset
        df_temp = pd.DataFrame(dict, index=[0])
        df = pd.concat([df, df_temp])

# change when run
df.to_csv("imageNetC_layer3.csv")

Building model...
Epoch 0, loss = 2.5174, lr 0.00100000
Got 777 / 2000 correct (38.85)

Epoch 1, loss = 1.2599, lr 0.00100000
Got 911 / 2000 correct (45.55)

Epoch 2, loss = 0.6180, lr 0.00100000
Got 918 / 2000 correct (45.90)

Epoch 3, loss = 0.2792, lr 0.00100000
Got 944 / 2000 correct (47.20)

Epoch 4, loss = 0.4151, lr 0.00100000
Got 943 / 2000 correct (47.15)

Epoch 5, loss = 0.4830, lr 0.00100000
Got 908 / 2000 correct (45.40)

Epoch 6, loss = 0.2613, lr 0.00100000
Got 884 / 2000 correct (44.20)

Epoch 7, loss = 0.1936, lr 0.00100000
Got 900 / 2000 correct (45.00)

Epoch 8, loss = 0.1655, lr 0.00100000
Got 903 / 2000 correct (45.15)

Epoch 9, loss = 0.3072, lr 0.00100000
Got 908 / 2000 correct (45.40)

Building model...
Epoch 0, loss = 3.2723, lr 0.00010000
Got 995 / 2000 correct (49.75)

Epoch 1, loss = 1.7378, lr 0.00010000
Got 1078 / 2000 correct (53.90)

Epoch 2, loss = 2.3462, lr 0.00010000
Got 1078 / 2000 correct (53.90)

Epoch 3, loss = 0.9191, lr 0.00010000
Got 1072 / 200

In [6]:
# four layers to train

for corruption in corruption_list:
    for lr in lrs:
        dataset = ImageNetC(root='/Users/rada/Desktop/KAIST/Deep Learning/Datasets/ImageNetC',
                        corruption=corruption, severity=5,
                        transform=T.Compose([T.ToTensor()])
                        )

        indices = torch.randperm(len(dataset))

        ictrain = torch.utils.data.Subset(dataset, indices[:n_subset])
        icvalid = torch.utils.data.Subset(dataset, indices[n_subset:n_subset+2000])
        ictest = torch.utils.data.Subset(dataset, indices[n_subset+2000:n_subset+4000])

        ictrainload = DataLoader(ictrain, batch_size=64, shuffle=True)
        icvalidload = DataLoader(icvalid, batch_size=64, shuffle=True)
        ictestload = DataLoader(ictest, batch_size=64, shuffle=True)

        
        device = torch.device("mps")
        tune_net = build_resnet50(device)

        # tune_net.layer1 to tune the first layer, tune_net to tune all layers, tune_net.fc to tune the last year
        optimizer = optim.Adam(tune_net.layer4.parameters(), lr=lr, weight_decay=0.0001)

        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
        
        # checkpoint = torch.load('checkpoints/ckpt-2.pth')
        # tune_net.load_state_dict(checkpoint['net'])

        acc = train_imagenetc(tune_net, ictrainload, icvalidload, optimizer, scheduler, device=device, epochs=10)

        # adding results to dataframe
        dict = {}
        dict['corruption'] = corruption
        dict['accuracy'] = acc
        dict['lr'] = lr
        dict['state'] = "fourth layer" # change when run
        dict['n_subset'] = n_subset
        df_temp = pd.DataFrame(dict, index=[0])
        df = pd.concat([df, df_temp])

# change when run
df.to_csv("imageNetC_layer4.csv")

Building model...
Epoch 0, loss = 2.7801, lr 0.00100000
Got 632 / 2000 correct (31.60)

Epoch 1, loss = 0.4444, lr 0.00100000
Got 689 / 2000 correct (34.45)

Epoch 2, loss = 0.1856, lr 0.00100000
Got 691 / 2000 correct (34.55)

Epoch 3, loss = 0.0585, lr 0.00100000
Got 706 / 2000 correct (35.30)

Epoch 4, loss = 0.0240, lr 0.00100000
Got 695 / 2000 correct (34.75)

Epoch 5, loss = 0.0820, lr 0.00100000
Got 693 / 2000 correct (34.65)

Epoch 6, loss = 0.0976, lr 0.00100000
Got 670 / 2000 correct (33.50)

Epoch 7, loss = 0.0516, lr 0.00100000
Got 636 / 2000 correct (31.80)

Epoch 8, loss = 0.1697, lr 0.00100000
Got 664 / 2000 correct (33.20)

Epoch 9, loss = 0.2886, lr 0.00100000
Got 644 / 2000 correct (32.20)

Building model...
Epoch 0, loss = 3.8143, lr 0.00010000
Got 930 / 2000 correct (46.50)

Epoch 1, loss = 2.5287, lr 0.00010000
Got 980 / 2000 correct (49.00)

Epoch 2, loss = 2.0801, lr 0.00010000
Got 981 / 2000 correct (49.05)

Epoch 3, loss = 0.4972, lr 0.00010000
Got 975 / 2000 c

In [7]:
# four layers to train

for corruption in corruption_list:
    for lr in lrs:
        dataset = ImageNetC(root='/Users/rada/Desktop/KAIST/Deep Learning/Datasets/ImageNetC',
                        corruption=corruption, severity=5,
                        transform=T.Compose([T.ToTensor()])
                        )

        indices = torch.randperm(len(dataset))

        ictrain = torch.utils.data.Subset(dataset, indices[:n_subset])
        icvalid = torch.utils.data.Subset(dataset, indices[n_subset:n_subset+2000])
        ictest = torch.utils.data.Subset(dataset, indices[n_subset+2000:n_subset+4000])

        ictrainload = DataLoader(ictrain, batch_size=64, shuffle=True)
        icvalidload = DataLoader(icvalid, batch_size=64, shuffle=True)
        ictestload = DataLoader(ictest, batch_size=64, shuffle=True)

        
        device = torch.device("mps")
        tune_net = build_resnet50(device)

        # tune_net.layer1 to tune the first layer, tune_net to tune all layers, tune_net.fc to tune the last year
        optimizer = optim.Adam(tune_net.fc.parameters(), lr=lr, weight_decay=0.0001)

        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
        
        # checkpoint = torch.load('checkpoints/ckpt-2.pth')
        # tune_net.load_state_dict(checkpoint['net'])

        acc = train_imagenetc(tune_net, ictrainload, icvalidload, optimizer, scheduler, device=device, epochs=10)

        # adding results to dataframe
        dict = {}
        dict['corruption'] = corruption
        dict['accuracy'] = acc
        dict['lr'] = lr
        dict['state'] = "fully-connected layer" # change when run
        dict['n_subset'] = n_subset
        df_temp = pd.DataFrame(dict, index=[0])
        df = pd.concat([df, df_temp])

# change when run
df.to_csv("imageNetC_fclayer.csv")

Building model...
Epoch 0, loss = 3.9903, lr 0.00100000
Got 708 / 2000 correct (35.40)

Epoch 1, loss = 1.1598, lr 0.00100000
Got 762 / 2000 correct (38.10)

Epoch 2, loss = 1.6377, lr 0.00100000
Got 742 / 2000 correct (37.10)

Epoch 3, loss = 1.1191, lr 0.00100000
Got 720 / 2000 correct (36.00)

Epoch 4, loss = 1.4310, lr 0.00100000
Got 705 / 2000 correct (35.25)

Epoch 5, loss = 0.7238, lr 0.00100000
Got 710 / 2000 correct (35.50)

Epoch 6, loss = 1.0443, lr 0.00100000
Got 692 / 2000 correct (34.60)

Epoch 7, loss = 0.5720, lr 0.00100000
Got 682 / 2000 correct (34.10)

Epoch 8, loss = 0.3300, lr 0.00100000
Got 688 / 2000 correct (34.40)

Epoch 9, loss = 0.2247, lr 0.00100000
Got 668 / 2000 correct (33.40)

Building model...
Epoch 0, loss = 4.0469, lr 0.00010000
Got 871 / 2000 correct (43.55)

Epoch 1, loss = 2.5612, lr 0.00010000
Got 938 / 2000 correct (46.90)

Epoch 2, loss = 3.4104, lr 0.00010000
Got 928 / 2000 correct (46.40)

Epoch 3, loss = 3.2764, lr 0.00010000
Got 916 / 2000 c

In [8]:
# four layers to train

for corruption in corruption_list:
    for lr in lrs:
        dataset = ImageNetC(root='/Users/rada/Desktop/KAIST/Deep Learning/Datasets/ImageNetC',
                        corruption=corruption, severity=5,
                        transform=T.Compose([T.ToTensor()])
                        )

        indices = torch.randperm(len(dataset))

        data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

        ictrain = torch.utils.data.Subset(dataset, indices[:n_subset])
        icvalid = torch.utils.data.Subset(dataset, indices[n_subset:n_subset+2000])
        ictest = torch.utils.data.Subset(dataset, indices[n_subset+2000:n_subset+4000])

        ictrainload = DataLoader(ictrain, batch_size=64, shuffle=True)
        icvalidload = DataLoader(icvalid, batch_size=64, shuffle=True)
        ictestload = DataLoader(ictest, batch_size=64, shuffle=True)

        
        device = torch.device("mps")
        tune_net = build_resnet50(device)

        # tune_net.layer1 to tune the first layer, tune_net to tune all layers, tune_net.fc to tune the last year
        optimizer = optim.Adam(tune_net.parameters(), lr=lr, weight_decay=0.0001)

        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
        
        # checkpoint = torch.load('checkpoints/ckpt-2.pth')
        # tune_net.load_state_dict(checkpoint['net'])

        acc = train_imagenetc(tune_net, ictrainload, icvalidload, optimizer, scheduler, device=device, epochs=10)

        # adding results to dataframe
        dict = {}
        dict['corruption'] = corruption
        dict['accuracy'] = acc
        dict['lr'] = lr
        dict['state'] = "all layers" # change when run
        dict['n_subset'] = n_subset
        df_temp = pd.DataFrame(dict, index=[0])
        df = pd.concat([df, df_temp])

# change when run
df.to_csv("imageNetC_allLayers.csv")

Building model...
Epoch 0, loss = 3.6219, lr 0.00100000
Got 355 / 2000 correct (17.75)

Epoch 1, loss = 1.8469, lr 0.00100000
Got 411 / 2000 correct (20.55)

Epoch 2, loss = 0.7686, lr 0.00100000
Got 400 / 2000 correct (20.00)

Epoch 3, loss = 0.2102, lr 0.00100000
Got 441 / 2000 correct (22.05)

Epoch 4, loss = 0.0307, lr 0.00100000
Got 411 / 2000 correct (20.55)

Epoch 5, loss = 0.3303, lr 0.00100000
Got 426 / 2000 correct (21.30)

Epoch 6, loss = 0.2443, lr 0.00100000
Got 393 / 2000 correct (19.65)

Epoch 7, loss = 0.1276, lr 0.00100000
Got 386 / 2000 correct (19.30)

Epoch 8, loss = 0.3241, lr 0.00100000
Got 290 / 2000 correct (14.50)

Epoch 9, loss = 0.5907, lr 0.00100000
Got 265 / 2000 correct (13.25)

Building model...
Epoch 0, loss = 2.1077, lr 0.00010000
Got 1003 / 2000 correct (50.15)

Epoch 1, loss = 0.6078, lr 0.00010000
Got 1066 / 2000 correct (53.30)

Epoch 2, loss = 0.5411, lr 0.00010000
Got 1050 / 2000 correct (52.50)

Epoch 3, loss = 0.2239, lr 0.00010000
Got 1051 / 20